In [4]:
import pandas
import re

from time import time

from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

url = 'data-pre-processing.csv'
dataframe = pandas.read_csv(url)

tokenizer = RegexpTokenizer("[\w']+")
stemmer = PorterStemmer()
vectorizer = TfidfVectorizer(stop_words='english')

corpus = []
corpusClassification = []

# print("Removing contraction - replacer.replace")
# print("Removing special chars - re.sub") -> Pay attention to <br> </br> (ouseébrHEUehUE)
# print("Steeming words - steemer.stem")
for videoId,author,date,content,classification in dataframe.values:
    comment = []
    content = content.lower()
    content = re.sub('(\s|\<)br\s*.\>', '', content)
    content = re.sub('(href.*)?http(.*)(\.com)?', 'HTTPWEBSITE', content)
    content = re.sub('[^A-Za-z0-9\s]+', '', content)
    corpusClassification.append(classification)
    for word in tokenizer.tokenize(content):
            word = stemmer.stem(word)
            comment.append(word)
    corpus.append(" ".join(comment))

print("Creating bag of words - vectorizer.fit_transform")
print()

vectorized =  vectorizer.fit_transform(corpus)

Creating bag of words - vectorizer.fit_transform



In [14]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn import metrics
import matplotlib.pyplot as plt

silhouetteValues = []
maxK = 4

print("Starting K-means")
# for k in range(2, maxK):
#     km = KMeans(n_clusters=k, init='k-means++', max_iter=1000, verbose=False)
#     t0 = time()
#     km.fit(vectorized.toarray())
#     labels = km.labels_
#     silhouette = metrics.silhouette_score(vectorized.toarray(), labels, metric='euclidean')
#     silhouetteValues.append(silhouette)
#     print("Clusters: %d - %0.5f - done in %0.3fs" % (k, silhouette, (time() - t0)))
#     true_k = k

# plt.title('Silhouette')
# plt.plot(list(range(2,maxK)), silhouetteValues)

# plt.show()

true_k = 2
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=5000, verbose=False)
km.fit(vectorized.toarray())
labels = km.labels_
# result = km.predict(vectorized.toarray())
rand = adjusted_rand_score(corpusClassification, labels)
print(rand)

print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Starting K-means
0.0128636375328
Top terms per cluster:
Cluster 0: httpwebsit song love thi subscrib like check channel pleas video
Cluster 1: youtub check video thi playlist look tri like httpwebsit pleas


In [22]:
from sklearn.cluster import AgglomerativeClustering

print("Compute structured hierarchical clustering...")

n_clusters = 10  # number of regions
ward = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')

t0 = time()
ward.fit(vectorized.toarray())
print("done in %0.3fs" % (time() - t0))

print()
print("Top terms per cluster:")
order_centroids = ward.children_
terms = vectorizer.get_feature_names()
print(ward.children_)
print(ward.labels_ )
for i in range(n_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i]:
        print(' %s' % terms[ind], end='')
    print()

Compute structured hierarchical clustering...
done in 8.970s

Top terms per cluster:
[[ 134  159]
 [ 585 1956]
 [ 825 1957]
 ..., 
 [2107 3801]
 [3882 3907]
 [3908 3909]]
[0 0 0 ..., 0 8 8]
Cluster 0: 500 753
Cluster 1: chack life
Cluster 2: datpiffcommixtapesdetailphpid633807 life39
Cluster 3: dispos lift
Cluster 4: frigea light
Cluster 5: hate lik
Cluster 6: hrefhttpssoundcloudcomroccsteadywaveemojiprodbynippylongbottomcyberpunkhttpssoundcloudcomroccsteadywaveemojiprodbynippylongbottomcyberpunka like
Cluster 7: httpminhatecacombrmaurosp2013filmesseriesdesenhosanimesmp3etc likeamp
Cluster 8: httpshhortcomaroocnjqu2b likebr
Cluster 9: jaylan likecom


In [42]:
comment = "z13rv5wie br u2fdnogl04cdl0bdmjuunpwcnk0k,C Williams,2015-05-18T21:51:36.079000,Check out this playlist on YouTube<br />ï»¿,1"

comment = re.sub('(\s|\<)br\s*.\>', '', comment)

print(comment)

z13rv5wie br u2fdnogl04cdl0bdmjuunpwcnk0k,C Williams,2015-05-18T21:51:36.079000,Check out this playlist on YouTubeï»¿,1
